In [1]:
import numpy as np
import pandas as pd
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, DBSCAN

In [ ]:

param_dic = {
    "host"      : "localhost",
    "database"  : "ml_fundamentus",
    "user"      : "postgres",
    "password"  : "postgres"
}
def connect(params_dic):
    conn = None
    try:

        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    

    tupples = cursor.fetchall()
    cursor.close()
    
    
    df = pd.DataFrame(tupples, columns=column_names)
    return df
 

conn = connect(param_dic)
column_names = ['id','papel','cotacao','p_l','p_vp','psr','div_yield','p_ativo','p_cap_giro','p_ebit','p_ativo_circulante','ev_ebit','ev_ebitda','mrg_ebit','mrg_liquida','liq_corr','roic','roe','liquides_2meses','patrim_liquido','div_bruto_patrimonio','crescimento_recorente_5a']
# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, "select * from acao", column_names)

print(df.head())

In [ ]:
df=df.drop('id', axis=1)  
df=df.drop('papel', axis=1)  

df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [6]:
#sns.pairplot(df)

In [7]:
X = df.values

In [8]:
km_2 = KMeans(n_clusters=2).fit(X)
km_3 = KMeans(n_clusters=3).fit(X)

In [9]:
km_2_labels = km_2.labels_
km_3_labels = km_3.labels_

iris_df_km_2 = df.copy()
iris_df_km_2['labels'] = km_2_labels

iris_df_km_3 = df.copy()
iris_df_km_3['labels'] = km_3_labels

In [ ]:
sns.pairplot(iris_df_km_2, hue='labels')

In [11]:
lista_de_WSS = []
lista_de_silhueta = []

for i in range(2, 15):
    km = KMeans(n_clusters=i).fit(X)
    lista_de_WSS.append(km.inertia_)
    lista_de_silhueta.append(silhouette_score(X, km.labels_))


In [ ]:
plt.plot(np.arange(2, 15), lista_de_WSS)

In [ ]:
plt.plot(np.arange(2, 15), lista_de_silhueta)